<a href="https://colab.research.google.com/github/TABARU-muu/2023-1Project/blob/main/_13_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommender_System)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [34]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


간단한 surprise 실습

In [35]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [36]:
data = Dataset.load_builtin('ml-100k',prompt=False)
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [37]:
model = SVD()

In [38]:
cross_validate(model,data,measures=['rmse','mae'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9383  0.9298  0.9351  0.9332  0.9418  0.9357  0.0041  
MAE (testset)     0.7372  0.7285  0.7395  0.7367  0.7437  0.7371  0.0050  
Fit time          1.44    0.96    0.94    0.95    0.91    1.04    0.20    
Test time         0.23    0.27    0.20    0.12    0.26    0.22    0.05    


{'test_rmse': array([0.93827915, 0.92981644, 0.93514368, 0.93323707, 0.94181056]),
 'test_mae': array([0.73721083, 0.72849545, 0.73946867, 0.73665844, 0.74373825]),
 'fit_time': (1.4409847259521484,
  0.9589171409606934,
  0.9425535202026367,
  0.9506816864013672,
  0.9094822406768799),
 'test_time': (0.23056364059448242,
  0.27413058280944824,
  0.20145773887634277,
  0.11857986450195312,
  0.25719714164733887)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [39]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [40]:
data = Dataset.load_builtin('ml-100k',prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [41]:
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [42]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users + 1,n_movies + 1)
shape

(943, 1682)

In [43]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id, movie_id,ratingg,time in raw_data:
  adj_matrix[user_id][movie_id] = 1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [44]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    similarity = np.dot(my_vector,user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector


print('Bset Match: {}, Best Match ID:{}'.format(best_match,best_match_id))

Bset Match: 183, Best Match ID:275


In [45]:
recommend_list=[]
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1,log2 = log
  if log1 < 1.and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [56]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector


print('Bset Match: {}, Best Match ID:{}'.format(best_match,best_match_id))

Bset Match: 0.0, Best Match ID:1


In [54]:
recommend_list=[]
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1,log2 = log
  if log1 < 1.and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [48]:
def compute_cos_similarity(v1,v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1,v2)
  return dot /(norm1*norm2)

In [52]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector


print('Bset Match: {}, Best Match ID:{}'.format(best_match,best_match_id))

Bset Match: 0.5278586163659506, Best Match ID:915


In [58]:
recommend_list=[]
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1,log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

Best Match: 55.06359959174482, Best Match ID: 737


Best Match: 0.569065731527988, Best Match ID: 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

* KNN을 사용한 협업 필터링

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9738  0.9806  0.9880  0.9769  0.9746  0.9788  0.0052  
MAE (testset)     0.7699  0.7722  0.7797  0.7728  0.7693  0.7728  0.0037  
Fit time          0.56    0.78    0.76    1.12    0.76    0.80    0.18    
Test time         8.19    9.94    9.41    7.25    5.05    7.97    1.74    


* SVD를 사용한 협업 필터링

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9372  0.9371  0.9364  0.9398  0.9365  0.9374  0.0012  
MAE (testset)     0.7395  0.7387  0.7361  0.7414  0.7383  0.7388  0.0017  
Fit time          15.97   18.80   19.45   16.82   12.28   16.67   2.53    
Test time         0.84    0.60    0.43    0.27    0.18    0.47    0.24    


* NMF를 사용한 협업 필터링

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9630  0.9628  0.9533  0.9643  0.9592  0.9605  0.0040  
MAE (testset)     0.7562  0.7559  0.7497  0.7589  0.7567  0.7555  0.0031  
Fit time          14.35   18.12   17.80   15.44   10.54   15.25   2.75    
Test time         0.67    0.49    0.39    0.24    0.14    0.39    0.18    


* SVD++를 사용한 협업 필터링

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9237  0.9107  0.9172  0.9213  0.9272  0.9200  0.0057  
MAE (testset)     0.7230  0.7136  0.7205  0.7230  0.7278  0.7216  0.0047  
Fit time          686.16  675.43  688.32  673.90  172.86  579.34  203.32  
Test time         12.88   12.85   8.31    12.56   3.16    9.95    3.81    


## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

(943, 2)
(2, 2)
(2, 1682)


array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

Best Match: 0.9999942295956324, Best Match ID: 235


[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

Best Match: 0.9999999951364141, Best Match ID: 1287


[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_nmf.py:1035: FutureWarning: The default value of init will change from random to None in 0.23 to make it consistent with decomposition.NMF.
  "with decomposition.NMF.", FutureWarning)


array([[3.71104386e+00, 1.48478414e+00, 7.39554801e-01, ...,
        3.64526020e-03, 1.45529217e-02, 1.44127541e-02],
       [2.11740926e+00, 2.36733594e-01, 5.51609827e-01, ...,
        4.76715033e-03, 2.44095997e-05, 0.00000000e+00],
       [9.85437620e-01, 1.10175294e-01, 2.56718002e-01, ...,
        2.21862129e-03, 1.13601741e-05, 0.00000000e+00],
       ...,
       [1.04462665e+00, 1.16792830e-01, 2.72137435e-01, ...,
        2.35187989e-03, 1.20425082e-05, 0.00000000e+00],
       [1.45791701e+00, 5.42098835e-01, 2.99293177e-01, ...,
        1.61349670e-03, 5.15831117e-03, 5.10699643e-03],
       [2.44660591e+00, 9.41428352e-01, 4.95526147e-01, ...,
        2.56812371e-03, 9.08641597e-03, 8.99741614e-03]])

* 사용자 기반 추천

In [ ]:
my_id, my_vector = 0, U[0]  # 내 id가 0번 이라고 가정
b

Best Match: 0.99999996802728, Best Match ID: 166


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[287, 289, 317, 363, 380, 391, 403, 434, 464, 477, 485, 492, 511, 512, 520, 529, 553, 567, 602, 605, 614, 640, 654, 658, 672, 673, 674, 697, 718, 725, 732, 734, 830, 948, 1124, 1125, 1146, 1199, 1224, 1303, 1304, 1305, 1306, 1307, 1308, 1309]


* 항목 기반 추천

In [ ]:
my_id, my_vector = 0, B.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(V.T):
  if my_id != item_id:
    cos_similarity = compute_cos_similarity(my_vector, item_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = item_id
      best_match_vector = item_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999346987367, Best Match ID: 497


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 